# Creating a Map displaying all the flights I have ever taken and more....

# Trying to add heads


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LinearSegmentedColormap, PowerNorm
import os
import matplotlib.animation as animation
from matplotlib.lines import Line2D

## sort the filenames out
cwd = os.getcwd()
print cwd
in_filename = cwd + '\Data\SW-J flight travel.csv'
print in_filename
    
# define the expected CSV columns
csv_cols = ('flight', 'dep_coutry', 'dep_airport','dep_airport_code','dep_lat','dep_lon',
                'arr_country','arr_airport','arr_airport_code','arr_lat','arr_lon',
                'date', 'notes','trip_name')

##clour set up
bg_color = (0.0, 0.0, 0, 1.0)
coast_color = (204/255.0, 0, 193/255.0, .9)
#coast_color = (204/255.0, 0, 153/255.0, 0.7)
color = (1.0, 0.80000000000000004, 0.90196078431372551, 1.0)

#set initial conditions
m = Basemap(projection='mill', lon_0=0, resolution= 'l')
m.drawcoastlines(color=coast_color, linewidth=1.5)
m.fillcontinents(color=bg_color, lake_color=bg_color)
m.drawmapboundary(fill_color=bg_color)
lines = []


# Load in the file 
routes = pd.read_csv(in_filename,names= csv_cols, skiprows=1)


#Populated the lines list
for i, route in enumerate(routes.sort_values(by='flight',
                              ascending=True).iterrows()):
    route = route[1]
    line, = m.drawgreatcircle(route['dep_lon'], route['dep_lat'],
                                  route['arr_lon'], route['arr_lat'],
                                  linewidth=2.0, color= color)
    lines.append(line)  



##set up the animation figure
current_line = 0
current_point = 0
fig2 = plt.figure(figsize=(27, 20))
m2 = Basemap(projection='mill', lon_0=0, resolution= 'l')
m2.drawcoastlines(color=coast_color, linewidth=1.5)
m2.fillcontinents(color=bg_color, lake_color=bg_color)
m2.drawmapboundary(fill_color=bg_color)

                   

#initialise the flight path

x,y = lines[current_line].get_data()
x= x[0]
y= y[0]
flight_path = [[],[],[]]
for i in iter(range(len(lines))):
        line = m2.plot(x,y, color=color)[0]
        line_head = m2.plot(x,y, color='red', marker='o', markeredgecolor='r')[0]
        line_head_slice = m2.plot(x,y, color='red', linewidth=2)[0]
        line.set_data(x, y)
        flight_path[0].append(line)
        flight_path[1].append(line_head)           #head
        flight_path[2].append(line_head_slice)     #head slice
        

# used if repeat set to true
def init():
    #print "data is", repr(iter(range(len(lines))))
    global  current_line , current_point, flight_path
    current_line = 0
    current_point = 0
    x,y = lines[current_line].get_data()
    x= x[0]
    y= y[0]
    for i in iter(range(len(lines))):
        flight_path[0][i].set_data(x,y)
        flight_path[1][i].set_data(x,y)       #head
        flight_path[2][i].set_data(x,y)    #head slice
    return flight_path[0] + flight_path[1] + flight_path[2]


#how many frames do we need ??
s=0
for i in range(len(lines)):
    s += len(lines[i].get_data()[0])
print 'frame count = ' + str(s)

r = 0
##define the animation function
def animate(frames):
#def animate(frames, *args, **kwargs):
    global  current_line , current_point, flight_path, lines,r
    head_len = 10
    x,y = lines[current_line].get_data()
    
    flight_path[0][current_line].set_data(x[:current_point+1],y[:current_point+1])
    flight_path[1][current_line].set_data(x[current_point],y[current_point])
    if (current_point < 10):
        
        flight_path[2][current_line].set_data(x[:current_point+1],y[:current_point+1])
        
    if (current_point > 9):
        flight_path[2][current_line].set_data(x[(current_point-head_len):current_point],
                                              y[(current_point-head_len):current_point])    
        
    
    if (current_point == lines[current_line].get_data()[0].size -1):
        
        current_point = 0
        flight_path[2][current_line].set_data([],[])
        current_line += 1
        
    else:
        current_point += 1   
    r += 1
    return flight_path[0] + flight_path[1] + flight_path[2]
            
#fargs= (current_line, current_point, flight_path, lines)
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig2, animate, frames=s-1, interval=5, blit=True, init_func=init, repeat= False)
print r

##movie stuff 
# Uncomment to save
ffmpeg_path = cwd + '\\ffmpeg-20160924-267da70-win64-static\\bin'
plt.rcParams['animation.ffmpeg_path'] = ffmpeg_path + '\\ffmpeg'
#mywriter = animation.FFMpegWriter(fps = 70)
#anim.save('mymovie.mp4',writer=mywriter)
plt.show()

C:\Users\vz28\Mapping project\python mapping
C:\Users\vz28\Mapping project\python mapping\Data\SW-J flight travel.csv
frame count = 2177
0


# that's it for now folks

In [26]:
#plt.rcParams
# animation.ImageMagickWriter.isAvailable()